# Ollama

In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
35 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [ ]:
# !ollama pull deepseek-r1:1.5b  # or your preferred model
# !ollama pull nomic-embed-text #
# !ollama pull llama3.2:3b
!ollama pull llama3.1:8b
!ollama pull mistral:7b
# !ollama pull paraphrase-multilingual
# !ollama pull gemma3:12b


# Installing more package

In [ ]:
# Clone the repository if it doesn't exist
# !git clone https://github.com/daoanhkhoa123/SupertCagRag
# %cd /kaggle/working/SupertCagRag

# Install required packages from requirements.txt
!pip install -q langchain langchain-ollama ollama
! pip install -q tqdm
# Update system packages and install additional tools
# !apt-get update -q
# !apt-get install -y -q poppler-utils

# !pip install -U -q langchain-community
# !pip install -qU transformers

# !pip install -q git+https://github.com/huggingface/transformers.git qwen-vl-utils flash-attn optimum auto-gptq bitsandbytes
# !pip install -q ollama
# !pip install -q colab-xterm
# !sudo apt-get update
# !sudo apt-get install poppler-utils

 # Code

In [ ]:
TEMPLATE = """
You are given:
- A student's subject scores (0–1000 scale) with these interpretations and answer guidelines:

  < 350: Very weak understanding
    → Use simple, everyday language and short sentences.
    → Avoid jargon; if unavoidable, define it simply.
    → Focus on core concepts only; give concrete examples.
    → Avoid technical depth; keep explanations step-by-step.

  350–500: Incomplete understanding
    → Use mostly simple language, introducing basic technical terms with definitions.
    → Explain the main concepts and relationships without overwhelming detail.
    → Use analogies and short examples.
    → Avoid advanced formulas or edge cases.

  500–650: Average understanding
    → Use standard technical vocabulary but explain less common terms.
    → Include moderate detail and simple examples.
    → Focus on solidifying fundamentals while gently introducing intermediate ideas.
    → Keep explanations structured and concise.

  650–800: Good understanding
    → Use precise technical terminology without over-explaining common concepts.
    → Include intermediate to advanced details and discuss key nuances.
    → Use examples and small real-world applications.
    → Briefly touch on complexities and edge cases.

  800–930: Strong understanding
    → Use advanced technical vocabulary and concise explanations.
    → Dive into deeper reasoning, implications, and trade-offs.
    → Use fewer examples unless illustrating complex points.
    → Highlight connections to broader concepts or related topics.

  930–1000: Excellent understanding
    → Use expert-level technical detail and terminology.
    → Discuss advanced nuances, optimizations, and theoretical aspects.
    → Avoid over-explaining basics; focus on high-level insights and subtle points.
    → Include references to related advanced concepts or frameworks.

- A technical document containing the reference content.
- A question to answer.

Your task:
- Tailor the answer according to the student’s score range rules above.
- Focus on concepts the student is most likely to find challenging for their level.
- Ensure accuracy and base reasoning solely on the provided document context.
- Do not explain your reasoning process.
- Keep the answer under 300 tokens.

Inputs:
- Document Context: {context}
- Student Score Profile: {profile}
- Question: {question}

Output:
- The pedagogically adapted answer.
"""

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
import json

# Load model
model_name = "llama3.1:8b"
model = OllamaLLM(model=model_name)


# Build the prompt from template
prompt = ChatPromptTemplate.from_template(TEMPLATE)

# Create the chain
chain = prompt | model

# Function to run the chain
def answer_question(context, question, profile):
  return chain.invoke({"question": question, "context": context, "profile":profile})

In [ ]:
# answer_question("a", "b" ,"c")

# Run


In [ ]:
import pandas as pd
ref = pd.read_csv("/content/evaled_combineddata.csv")
# row = ref.copy()
# row

In [ ]:
# M ĐỔI THÀNH HEAD(50) Ở ĐÂY

df = ref.head(2)

df.shape

(2, 7)

In [ ]:
from tqdm import tqdm
import json

graded_results = []

# Loop through all entries in DataFrames
for idx, row in enumerate(tqdm(df.iterrows(), total=len(df), desc="Grading Responses")):
    row = row[1]

    try:
        res = answer_question(row["context"], row["question"], row["profile"])
    except Exception as e:
        print(f"❌ Error at row {idx}: {e}")
        continue

    if res is None:
        continue

    graded_results.append({
        "userid": row['userid'],
        "model": model_name,
        "profile": row['profile'],
        "context": row['context'],
        "question": row['question'],
        "answer_ref": row['answer'],
        "answer_eval": res,
    })

    # Save every 25 entries
    if (idx + 1) % 10 == 0:
        temp_filename = f"dataset_batch_{idx+1}.json"
        with open(temp_filename, "w", encoding="utf-8") as f:
            json.dump(graded_results, f, ensure_ascii=False, indent=2)
        print(f"✅ Saved batch to '{temp_filename}'")

# Final save with full dataset
with open(f"{model_name}_genanswers.json", "w", encoding="utf-8") as f:
    json.dump(graded_results, f, ensure_ascii=False, indent=2)

print(f"🎉 All responses graded and saved to '{model_name}_genanswers.json'")


Grading Responses: 100%|██████████| 2/2 [00:12<00:00,  6.17s/it]

🎉 All responses graded and saved to 'llama3.1:8b_genanswers.json'


In [ ]:
import pandas as pd

df_graded_results = pd.DataFrame(graded_results)
df_graded_results.to_csv(f"{model_name}_genanswers.csv", index=False)